## Overview
This notebook covers the 80% of reading and writing files in pandas.</br>
While there are a large number of options in the pandas API, this will focus on:
* reading a directory
* CSV
* Parquet
* JSON
* Excel


To use you need to have python installed and jupyterlab.  </br>
The code assumes you have a basic familiarity with python syntax and use.
## Packages Needed
* sys
* os
* pandas
* numpy
* json
* pyarrow

## Install & Import


In [23]:
import sys


!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install pyarrow
!{sys.executable} -m pip install openpyxl

'''
Using "!{sys.executable} -m pip install"   instead of "!pip install"
ensures that the install is done in the context and kernel currently running
the notebook. This is a recommended best practice and I try to use this method within
notebooks as I try to default to what I would want to see if I was collaborating with
a group.
'''
import os
import numpy as np
import pandas as pd
import json

## Files used

All files were downloaded and extracted in to a folder called "data" in the
same folder as this notbook "./data/*"

From Kaggle
* https://www.kaggle.com/datasets/jeffreybraun/chipotle-locations
    * chipotle_store.csv
    * us-states.json


From Github
* https://github.com/Teradata/kylo/tree/master/samples/sample-data/parquet
    * userdata1.parquet


## Looping files in a directory

In [2]:
# print names of files in a directory and return them as a list.
def return_files_as_list(directory):
    files = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file before printing and adding to the
        # file list
        if os.path.isfile(f):
            print(f)
            files.append(f)
    return files

In [3]:
return_files_as_list("./data")

./data/userdata3.parquet
./data/chipotle_stores.csv
./data/userdata2.parquet
./data/emails.parquet
./data/userdata1.parquet
./data/output.xlsx
./data/us-states.json
./data/chipotle.tsv


['./data/userdata3.parquet',
 './data/chipotle_stores.csv',
 './data/userdata2.parquet',
 './data/emails.parquet',
 './data/userdata1.parquet',
 './data/output.xlsx',
 './data/us-states.json',
 './data/chipotle.tsv']

## Read/Write CSV
docs:
 * https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
 * https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [4]:
chipotle_loc_df = pd.read_csv("./data/chipotle_stores.csv")
chipotle_loc_df.head()

,state,location,address,latitude,longitude
0,Alabama,Auburn,"346 W Magnolia Ave Auburn, AL 36832 US",32.606813,-85.487328
1,Alabama,Birmingham,"300 20th St S Birmingham, AL 35233 US",33.509721,-86.802756
2,Alabama,Birmingham,"3220 Morrow Rd Birmingham, AL 35235 US",33.595581,-86.647437
3,Alabama,Birmingham,"4719 Highway 280 Birmingham, AL 35242 US",33.422582,-86.698279
4,Alabama,Cullman,"1821 Cherokee Ave SW Cullman, AL 35055 US",34.154134,-86.841220


In [5]:
chipotle_loc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2629 entries, 0 to 2628
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   state      2629 non-null   object 
 1   location   2629 non-null   object 
 2   address    2629 non-null   object 
 3   latitude   2629 non-null   float64
 4   longitude  2629 non-null   float64
dtypes: float64(2), object(3)
memory usage: 102.8+ KB


In [6]:
chipotle_loc_df.to_csv("./data/chipotle.tsv", sep="\t", index=False)
return_files_as_list("./data")

./data/userdata3.parquet
./data/chipotle_stores.csv
./data/userdata2.parquet
./data/emails.parquet
./data/userdata1.parquet
./data/output.xlsx
./data/us-states.json
./data/chipotle.tsv


['./data/userdata3.parquet',
 './data/chipotle_stores.csv',
 './data/userdata2.parquet',
 './data/emails.parquet',
 './data/userdata1.parquet',
 './data/output.xlsx',
 './data/us-states.json',
 './data/chipotle.tsv']

In [7]:
tsv_df = pd.read_csv("./data/chipotle.tsv", sep="\t")
tsv_df.head()

,state,location,address,latitude,longitude
0,Alabama,Auburn,"346 W Magnolia Ave Auburn, AL 36832 US",32.606813,-85.487328
1,Alabama,Birmingham,"300 20th St S Birmingham, AL 35233 US",33.509721,-86.802756
2,Alabama,Birmingham,"3220 Morrow Rd Birmingham, AL 35235 US",33.595581,-86.647437
3,Alabama,Birmingham,"4719 Highway 280 Birmingham, AL 35242 US",33.422582,-86.698279
4,Alabama,Cullman,"1821 Cherokee Ave SW Cullman, AL 35055 US",34.154134,-86.841220


### Chunky chunks

In [8]:
'''
A method that is often taught too late or not at all is chunks.
for large data sets or memory constrained compute like laptops, it is
very helpful to learn chunk early.
'''

for chunk_df in pd.read_csv("./data/chipotle_stores.csv", chunksize=2):
    print(chunk_df.info())
    break


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   state      2 non-null      object 
 1   location   2 non-null      object 
 2   address    2 non-null      object 
 3   latitude   2 non-null      float64
 4   longitude  2 non-null      float64
dtypes: float64(2), object(3)
memory usage: 208.0+ bytes
None


## Read/Write Parquet

In [9]:
parquet_df = pd.read_parquet("./data/userdata1.parquet")
parquet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   registration_dttm  1000 non-null   datetime64[ns]
 1   id                 1000 non-null   int32         
 2   first_name         1000 non-null   object        
 3   last_name          1000 non-null   object        
 4   email              1000 non-null   object        
 5   gender             1000 non-null   object        
 6   ip_address         1000 non-null   object        
 7   cc                 1000 non-null   object        
 8   country            1000 non-null   object        
 9   birthdate          1000 non-null   object        
 10  salary             932 non-null    float64       
 11  title              1000 non-null   object        
 12  comments           994 non-null    object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(10)
memory us

In [10]:
parquet_df[["first_name", "email"]].to_parquet("./data/emails.parquet")
'''
This uses Pyarrow under the hood to serialize the data
 and save the file
'''
emails_df = pd.read_parquet("./data/emails.parquet")
emails_df.head()

,first_name,email
0,Amanda,ajordan0@com.com
1,Albert,afreeman1@is.gd
2,Evelyn,emorgan2@altervista.org
3,Denise,driley3@gmpg.org
4,Carlos,cburns4@miitbeian.gov.cn


## Read/Write JSON

In [11]:
json_df = pd.read_json("./data/us-states.json")
json_df.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'id': 'AL', 'properties': ..."
1,FeatureCollection,"{'type': 'Feature', 'id': 'AK', 'properties': ..."
2,FeatureCollection,"{'type': 'Feature', 'id': 'AZ', 'properties': ..."
3,FeatureCollection,"{'type': 'Feature', 'id': 'AR', 'properties': ..."
4,FeatureCollection,"{'type': 'Feature', 'id': 'CA', 'properties': ..."


In [12]:
json_data = json.load(open("./data/us-states.json"))
print(json.dumps(json_data, indent=2))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "id": "AL",
      "properties": {
        "name": "Alabama"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -87.359296,
              35.00118
            ],
            [
              -85.606675,
              34.984749
            ],
            [
              -85.431413,
              34.124869
            ],
            [
              -85.184951,
              32.859696
            ],
            [
              -85.069935,
              32.580372
            ],
            [
              -84.960397,
              32.421541
            ],
            [
              -85.004212,
              32.322956
            ],
            [
              -84.889196,
              32.262709
            ],
            [
              -85.058981,
              32.13674
            ],
            [
              -85.053504,
            

In [13]:
json_df2 = pd.DataFrame.from_records(json_data["features"])
json_df2.head()

,type,id,properties,geometry
0,Feature,AL,{'name': 'Alabama'},"{'type': 'Polygon', 'coordinates': [[[-87.3592..."
1,Feature,AK,{'name': 'Alaska'},"{'type': 'MultiPolygon', 'coordinates': [[[[-1..."
2,Feature,AZ,{'name': 'Arizona'},"{'type': 'Polygon', 'coordinates': [[[-109.042..."
3,Feature,AR,{'name': 'Arkansas'},"{'type': 'Polygon', 'coordinates': [[[-94.4738..."
4,Feature,CA,{'name': 'California'},"{'type': 'Polygon', 'coordinates': [[[-123.233..."


In [14]:
json_df3 = pd.DataFrame.from_dict(json_data["features"]) # same as from records above
json_df3 = pd.DataFrame.from_dict(json_data["features"][0]) # gets wonky

json_df3.head()

,type,id,properties,geometry
name,Feature,AL,Alabama,NaN
type,Feature,AL,NaN,Polygon
coordinates,Feature,AL,NaN,"[[[-87.359296, 35.00118], [-85.606675, 34.9847..."


## Read/Write Excel


In [15]:
parquet_df = pd.read_parquet("./data/userdata1.parquet")
parquet_df2 = pd.read_parquet("./data/userdata2.parquet")
parquet_df3 = pd.read_parquet("./data/userdata3.parquet")
parquet_df.head()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,
4,2016-02-03 05:05:31,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,NaN,,


In [16]:
parquet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   registration_dttm  1000 non-null   datetime64[ns]
 1   id                 1000 non-null   int32         
 2   first_name         1000 non-null   object        
 3   last_name          1000 non-null   object        
 4   email              1000 non-null   object        
 5   gender             1000 non-null   object        
 6   ip_address         1000 non-null   object        
 7   cc                 1000 non-null   object        
 8   country            1000 non-null   object        
 9   birthdate          1000 non-null   object        
 10  salary             932 non-null    float64       
 11  title              1000 non-null   object        
 12  comments           994 non-null    object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(10)
memory us

In [17]:
parquet_df.to_excel("./data/output.xlsx",
             sheet_name='user_data_1')

In [18]:
with pd.ExcelWriter('./data/output.xlsx') as writer:
    parquet_df.to_excel(writer, sheet_name='user_data_1')
    parquet_df2.to_excel(writer, sheet_name='user_data_2')
    parquet_df2.to_excel(writer, sheet_name='user_data_3')


In [19]:
excel_df = pd.read_excel("./data/output.xlsx", sheet_name="user_data_1")
excel_df.head()

,Unnamed: 0,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6.759522e+15,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,NaN,Canada,1/16/1968,150280.17,Accountant IV,NaN
2,2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6.767119e+15,Russia,2/1/1960,144972.51,Structural Engineer,NaN
3,3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3.576032e+15,China,4/8/1997,90263.05,Senior Cost Accountant,NaN
4,4,2016-02-03 05:05:31,5,Carlos,Burns,cburns4@miitbeian.gov.cn,NaN,169.113.235.40,5.602256e+15,South Africa,NaN,NaN,NaN,NaN


In [20]:
excel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Unnamed: 0         1000 non-null   int64         
 1   registration_dttm  1000 non-null   datetime64[ns]
 2   id                 1000 non-null   int64         
 3   first_name         984 non-null    object        
 4   last_name          1000 non-null   object        
 5   email              984 non-null    object        
 6   gender             933 non-null    object        
 7   ip_address         1000 non-null   object        
 8   cc                 709 non-null    float64       
 9   country            1000 non-null   object        
 10  birthdate          803 non-null    object        
 11  salary             932 non-null    float64       
 12  title              803 non-null    object        
 13  comments           179 non-null    object        
dtypes: dateti

## View all columns

In [21]:
pd.set_option('display.max_columns', None)

In [22]:
excel_df.head()


,Unnamed: 0,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,0,2016-02-03 07:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6.759522e+15,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,1,2016-02-03 17:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,NaN,Canada,1/16/1968,150280.17,Accountant IV,NaN
2,2,2016-02-03 01:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6.767119e+15,Russia,2/1/1960,144972.51,Structural Engineer,NaN
3,3,2016-02-03 00:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3.576032e+15,China,4/8/1997,90263.05,Senior Cost Accountant,NaN
4,4,2016-02-03 05:05:31,5,Carlos,Burns,cburns4@miitbeian.gov.cn,NaN,169.113.235.40,5.602256e+15,South Africa,NaN,NaN,NaN,NaN
